In [1]:
import requests
import csv
from bs4 import BeautifulSoup
import logging
from collections import namedtuple
import pickle
import multiprocessing
import time

from ufo_downloader import UFOData, LocationFinder

In [2]:
dataclass = UFOData('data.json')

In [3]:
finder = LocationFinder('location_cache.json')

In [4]:
for i, d in enumerate(dataclass.data):
    try:
        upd, http_use = finder.find_ugly(d['City'], d['State'],True)
        if http_use:
            time.sleep(0.7)
        dataclass.data[i].update(upd)
    except Exception as e:
        print(e)
        break
    if not (i % 1000):
        print(i)
    if not (i % 2000):
        finder.save_cache()
finder.save_cache()

0
Location cache size:18301
1000
2000
Location cache size:18301
3000
4000
Location cache size:18301
5000
6000
Location cache size:18301
7000
8000
Location cache size:18301
9000
10000
Location cache size:18301
11000
12000
Location cache size:18301
13000
14000
Location cache size:18301
15000
16000
Location cache size:18301
17000
18000
Location cache size:18301
19000
20000
Location cache size:18301
21000
22000
Location cache size:18301
23000
24000
Location cache size:18301
25000
26000
Location cache size:18301
27000
28000
Location cache size:18301
29000
30000
Location cache size:18301
31000
32000
Location cache size:18301
33000
34000
Location cache size:18301
35000
36000
Location cache size:18301
37000
38000
Location cache size:18301
39000
40000
Location cache size:18301
41000
42000
Location cache size:18301
43000
44000
Location cache size:18301
45000
46000
Location cache size:18301
47000
48000
Location cache size:18301
49000
50000
Location cache size:18301
51000
52000
Location cache size

In [ ]:
t = time.time()

def get_data_from_table(url:str,generate_desc:bool = False):
    soup = BeautifulSoup(requests.get(url).text,"html.parser")
    header = [i.text for i in  soup.thead.findAll("th")]
    data = []
    for row in soup.tbody.findAll("tr"):
        long_desc = ""
        if generate_desc:
            try:
                long_desc =  BeautifulSoup(requests.get(base_page+row.find("a").get("href")).text,"html.parser").tbody.findAll("td")[1].text
            except Exception as e:
                logger.log(level=logging.INFO,msg="could not retrieve long description")
        i = 0
        entry = {}
        for cell in row.findAll("td"):
            entry[header[i]] = cell.text
            i+=1
        entry["Description"] = long_desc
        data.append(entry)
    return data


# Entry = namedtuple("Entry",['Date / Time', 'City', 'State', 'Shape', 'Duration', 'Summary', 'Posted'])

bs = BeautifulSoup(resp.text,"html.parser")
for link in bs.table.tbody.findAll("a"):
    links_to_parse.append(base_page+link.get('href'))

    
#links_to_parse = ["http://www.nuforc.org/webreports/ndxschanged.html","http://www.nuforc.org/webreports/ndxsCrescent.html","http://www.nuforc.org/webreports/ndxsCross.html","http://www.nuforc.org/webreports/ndxsDelta.html","http://www.nuforc.org/webreports/ndxsDome.html","http://www.nuforc.org/webreports/ndxsFlare.html","http://www.nuforc.org/webreports/ndxsHexagon.html","http://www.nuforc.org/webreports/ndxspyramid.html","http://www.nuforc.org/webreports/ndxsRound.html"]

for link in links_to_parse:
    data = get_data_from_table(link,False)
    parsed_data.extend(data)
    print("done: "+link +" count: "+str(len(data)))
    
print(time.time()-t)
print("checking count, should be around 113241 : "+len(parsed_data))

csv.register_dialect("semicolon",delimiter=";")
with open("big_data.csv","w",newline="",encoding="utf-8") as f:
    dw = csv.DictWriter(f,parsed_data[0].keys(),dialect="semicolon")
    dw.writeheader()
    dw.writerows(parsed_data)


In [ ]:
with open("big_data.csv","w",newline="",encoding="utf-8") as f:
    dw = csv.DictWriter(f,parsed_data[0].keys(),dialect="semicolon")
    dw.writeheader()
    for item in parsed_data :
        try:
            dw.writerow(item)
        except Exception as e:
            print(item)

In [ ]:
with open("big_save.pckl","wb") as f:
    pickle.dump(parsed_data,f)


In [7]:
finder.save_cache()

Location cache size:11167
